<a href="https://colab.research.google.com/github/RasanaPJ/Deep-Learning/blob/master/imdb_review_using_wordembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

Load iimdb data set from tensorflow datasets, retrieve reviews along with its metadata info

In [0]:
import tensorflow_datasets as tfds

In [0]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [5]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=0.1.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [0]:
import numpy as np

The data set has total_num_examples=100000, with test: 25000 and train: 25000

In [0]:
imdb_train, imdb_test = imdb['train'], imdb['test']

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []
# Iterate over the training/test data and extract the trainig sentences and labels.
# The values of s and l are tensors hence use numpy() to extract their values
for s,l in imdb_train:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in imdb_test:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

Tokenize and pad sequence the training and test sentences

In [0]:
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok='<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok )
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequence = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequence, maxlen=max_length,truncating=trunc_type)

testing_sequence = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequence, maxlen=max_length,truncating=trunc_type)


Define a neural network

In [0]:
model= tf.keras.Sequential([
   tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length) ,
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(6,activation='relu'),
   tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [24]:
num_epochs=10
model.fit(padded,training_labels_final,epochs=num_epochs,verbose=1,validation_data=(testing_padded,testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


25000/25000 [==============================] - 7s 271us/sample - loss: 0.4909 - acc: 0.7460 - val_loss: 0.3782 - val_acc: 0.8310
Epoch 2/10
25000/25000 [==============================] - 6s 236us/sample - loss: 0.2547 - acc: 0.8993 - val_loss: 0.3977 - val_acc: 0.8260
Epoch 3/10
25000/25000 [==============================] - 6s 240us/sample - loss: 0.1224 - acc: 0.9639 - val_loss: 0.4915 - val_acc: 0.8086
Epoch 4/10
25000/25000 [==============================] - 6s 232us/sample - loss: 0.0368 - acc: 0.9944 - val_loss: 0.5863 - val_acc: 0.8078
Epoch 5/10
25000/25000 [==============================] - 6s 235us/sample - loss: 0.0091 - acc: 0.9989 - val_loss: 0.6796 - val_acc: 0.8051
Epoch 6/10
25000/25000 [==============================] - 6s 244us/sample - loss: 0.0026 - acc: 1.0000 - val_loss: 0.7530 - val_acc: 0.8057
Epoch 7/10
25000/25000 [==============================] - 6s 238us/sample - loss: 0.0011 - acc: 1.0000 - val_loss: 0.8082 - val_acc: 0.8053
Epoch 8/10
25000/25000 [=======

In [25]:
e = model.layers[0]
weights= e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [27]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b oh yeah jenna jameson did it again yeah baby this movie rocks it was one of the 1st movies i saw of her and i have to say i feel in love with her she was great in this move br br her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing br br i hope this comment helps and u can buy the movie the storyline is awesome is very unique and i'm sure u are going to like it jenna amazed us once more and no wonder the movie won so many
b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the

In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [30]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4738], [], [790], [2015], [11], [2922], [2191], [], [790], [2015], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2922], [1277], [], [], [2015], [1005], [2922], [968], [579], [790], []]


The above downloaded tsv files can be used to project the vetors in projector.tensorflow.org